In [1]:
import os
import sys
sys.path.append('../python')

import pandas as pd
import numpy as np
import yaml
import time
import api

DOCID = 'lamunicipalcode'


In [2]:
#data = pd.read_pickle(f"../../intermediate_data/{DOCID}_main.pkl")
#data = data.fillna('')
#data = data.loc[ data['text'].str.len() < 9000 ]  # Drop if text sizes are too large
#data = data.reset_index(names='id')
#data_to_embed = data[['id','doc_id','text_order','item_level','item_type','text']]
#
#api.embed_and_upsert(data_to_embed, namespace='main')

In [3]:
#data = pd.read_pickle(f"../../intermediate_data/{DOCID}_fullpassages.pkl")
#data = data.fillna('')
#data = data.loc[ data['text'].str.len() < 9000 ]  # Drop if text sizes are too large
#data = data.reset_index(names='id')
#data_to_embed = data[['id','doc_id','text_order','item_level','item_type','text']]
#
#api.embed_and_upsert(data_to_embed, namespace='fullpassages')

In [4]:
data = pd.read_pickle(f"../../intermediate_data/{DOCID}_passagechunks.pkl")
data = data.fillna('')
data = data.loc[ data['text'].str.len() < 9000 ]  # Drop if text sizes are too large
data = data.reset_index()
data_to_embed = data[['id','doc_id','start_id','end_id','chapter_id','article_id','section_id','item_type','text']]

api.embed_and_upsert(data_to_embed, namespace='passagechunks')

0... 128... 256... 384... 512... 640... 768... 896... 1024... 1152... 1280... 1408... 1536... 1664... 1792... 1920... 2048... 2176... 2304... 2432... 2560... 2688... 2816... 2944... 3072... 3200... 3328... 3456... 3584... 3712... 3840... 3968... 4096... 4224... 4352... 4480... 4608... 4736... 4864... 4992... 5120... 5248... 5376... 5504... 5632... 5760... 5888... 6016... 6144... 6272... 6400... 6528... 6656... 6784... 6912... 7040... 7168... 7296... 7424... 7552... 7680... 7808... 7936... 8064... 8192... 8320... 8448... 8576... 8704... {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'fullpassages': {'vector_count': 34586},
                'main': {'vector_count': 52869},
                'passagechunks': {'vector_count': 17246}},
 'total_vector_count': 104701}
